In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
from IPython.display import display

In [94]:
path_Manipulated = r"RealDataMerged\mergedBase.xlsx"

In [95]:
df_Manipulated = pd.read_excel(path_Manipulated)
display(df_Manipulated.head(5))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,20,2025-05-13 17:08:06,272.00,34.00,1.93,1.62,0.32
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,21,2025-01-22 07:21:28,265.36,33.17,0.87,0.52,0.35
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,22,2025-05-20 09:21:44,200.33,25.04,27.75,0.30,27.45
3,FA1114659,50,2024-05-06,2024-05-31,2024-05-31,2024-05-31,20,2025-01-17 07:02:59,178.52,22.32,0.97,0.35,0.62
4,FA1114659,50,2024-05-06,2024-05-31,2024-05-31,2024-05-31,40,2024-07-24 16:23:26,170.56,21.32,4.18,0.73,3.45


In [96]:
df_Manipulated['Startdatum'] = pd.to_datetime(df_Manipulated['Startdatum'])
df_Manipulated['Enddatum'] = pd.to_datetime(df_Manipulated['Enddatum'])

# Neue Spalte mit Differenz in Tagen --> entspricht Durchlaufzeit Soll
df_Manipulated['DurchlaufszeitSoll(t)'] = (df_Manipulated['Enddatum'] - df_Manipulated['Startdatum']).dt.days
display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t)
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,20,2025-05-13 17:08:06,272.00,34.00,1.93,1.62,0.32,9
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,21,2025-01-22 07:21:28,265.36,33.17,0.87,0.52,0.35,13
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,22,2025-05-20 09:21:44,200.33,25.04,27.75,0.30,27.45,13


In [98]:
df_Manipulated['Soll/Ist Durchlaufszeit(t)'] = (df_Manipulated['DurchlaufszeitSoll(t)'] - df_Manipulated['DurchlaufszeitIst(t)'])
display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t)
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,20,2025-05-13 17:08:06,272.00,34.00,1.93,1.62,0.32,9,-25.00
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,21,2025-01-22 07:21:28,265.36,33.17,0.87,0.52,0.35,13,-20.17
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,22,2025-05-20 09:21:44,200.33,25.04,27.75,0.30,27.45,13,-12.04


In [99]:
# Stelle sicher, dass beide Spalten im Datetime-Format sind
df_Manipulated['Fälligkeitsdatum'] = pd.to_datetime(df_Manipulated['Fälligkeitsdatum'])
df_Manipulated['Fertigstellungsdatum'] = pd.to_datetime(df_Manipulated['Fertigstellungsdatum'])

# Neue Spalte mit Abweichung in Tagen (positiv = verspätet, negativ = zu früh)
df_Manipulated['Abweichung (Tage)'] = (df_Manipulated['Fertigstellungsdatum'] - df_Manipulated['Fälligkeitsdatum']).dt.days

# Neue Spalte zur Bewertung der Termintreue
df_Manipulated['Termintreue'] = df_Manipulated['Abweichung (Tage)'].apply(
    lambda x: 0 if x <= 0 else 1)

display(df_Manipulated.head(3))

,FANummer,Menge,Startdatum,Enddatum,Fälligkeitsdatum,Fertigstellungsdatum,OperationNo,Logdatum,DurchlaufszeitIst(h),DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
0,FA1112305,21,2024-06-12,2024-06-21,2024-06-21,2024-06-26,20,2025-05-13 17:08:06,272.00,34.00,1.93,1.62,0.32,9,-25.00,5,1
1,FA1114090,300,2024-05-17,2024-05-30,2024-05-30,2024-05-31,21,2025-01-22 07:21:28,265.36,33.17,0.87,0.52,0.35,13,-20.17,1,1
2,FA1114511,36,2024-05-16,2024-05-29,2024-05-29,2024-05-29,22,2025-05-20 09:21:44,200.33,25.04,27.75,0.30,27.45,13,-12.04,0,0


In [100]:
df_Manipulated.drop(columns=['Startdatum', 'Enddatum', 'Fälligkeitsdatum', 'Fertigstellungsdatum', 'Logdatum', 'DurchlaufszeitIst(h)'], inplace=True)
display(df_Manipulated.head(5))

,FANummer,Menge,OperationNo,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
0,FA1112305,21,20,34.00,1.93,1.62,0.32,9,-25.00,5,1
1,FA1114090,300,21,33.17,0.87,0.52,0.35,13,-20.17,1,1
2,FA1114511,36,22,25.04,27.75,0.30,27.45,13,-12.04,0,0
3,FA1114659,50,20,22.32,0.97,0.35,0.62,25,2.68,0,0
4,FA1114659,50,40,21.32,4.18,0.73,3.45,25,3.68,0,0


In [101]:
df_Manipulated.describe(include='all')

,FANummer,Menge,OperationNo,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage),Termintreue
count,2894,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000,2894.000000
unique,1467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,FA1119067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,29.627851,75.863856,1.411818,1.352533,0.330681,1.022156,7.494126,6.082308,4.071527,0.658604
std,NaN,65.011866,251.840419,2.319995,2.643703,0.341881,2.534215,15.534307,15.591362,9.645249,0.474260
min,NaN,1.000000,10.000000,0.000000,-16.800000,-1.670000,-16.800000,0.000000,-25.000000,-41.000000,0.000000
25%,NaN,2.000000,20.000000,0.000000,0.320000,0.150000,0.070000,4.000000,2.000000,0.000000,0.000000
50%,NaN,10.000000,30.000000,0.610000,0.600000,0.300000,0.250000,6.000000,4.260000,2.000000,1.000000
75%,NaN,25.000000,40.000000,2.050000,1.300000,0.430000,0.837500,8.000000,7.000000,7.000000,1.000000


In [102]:
cols_merged = [
    'Menge',
    'DurchlaufszeitIst(t)',
    'Rüst-Bearbeitungszeit',
    'Rüstzeit',
    'Bearbeitungszeit',
    'DurchlaufszeitSoll(t)',
    'Soll/Ist Durchlaufszeit(t)',
    'Abweichung (Tage)'
    
]

corr_matrix = df_Manipulated[cols_merged].corr()
cov_matrix  = df_Manipulated[cols_merged].cov()

print("Korrelationsmatrix:")
display(corr_matrix.round(3))
print("\nKovarianzmatrix:")
display(cov_matrix.round(3))


Korrelationsmatrix:


,Menge,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage)
Menge,1.000,0.085,0.046,0.018,0.046,0.154,0.141,0.054
DurchlaufszeitIst(t),0.085,1.000,0.296,0.210,0.280,0.050,-0.099,-0.022
Rüst-Bearbeitungszeit,0.046,0.296,1.000,0.378,0.992,-0.003,-0.047,0.008
Rüstzeit,0.018,0.210,0.378,1.000,0.259,0.029,-0.002,-0.002
Bearbeitungszeit,0.046,0.280,0.992,0.259,1.000,-0.007,-0.048,0.009
DurchlaufszeitSoll(t),0.154,0.050,-0.003,0.029,-0.007,1.000,0.989,0.026
Soll/Ist Durchlaufszeit(t),0.141,-0.099,-0.047,-0.002,-0.048,0.989,1.000,0.029
Abweichung (Tage),0.054,-0.022,0.008,-0.002,0.009,0.026,0.029,1.000



Kovarianzmatrix:


,Menge,DurchlaufszeitIst(t),Rüst-Bearbeitungszeit,Rüstzeit,Bearbeitungszeit,DurchlaufszeitSoll(t),Soll/Ist Durchlaufszeit(t),Abweichung (Tage)
Menge,4226.543,12.779,7.899,0.393,7.504,155.352,142.573,33.757
DurchlaufszeitIst(t),12.779,5.382,1.813,0.166,1.646,1.803,-3.579,-0.494
Rüst-Bearbeitungszeit,7.899,1.813,6.989,0.341,6.647,-0.107,-1.920,0.203
Rüstzeit,0.393,0.166,0.341,0.117,0.224,0.154,-0.012,-0.007
Bearbeitungszeit,7.504,1.646,6.647,0.224,6.422,-0.261,-1.908,0.210
DurchlaufszeitSoll(t),155.352,1.803,-0.107,0.154,-0.261,241.315,239.511,3.942
Soll/Ist Durchlaufszeit(t),142.573,-3.579,-1.920,-0.012,-1.908,239.511,243.091,4.436
Abweichung (Tage),33.757,-0.494,0.203,-0.007,0.210,3.942,4.436,93.031


In [103]:
output_xlsx = r"RealDataMerged/mergedManupulated.xlsx"
df_Manipulated.to_excel(output_xlsx, index=False)